In [1]:
import pandas as pd
import geopandas as gpd

In [3]:
final_df = merge_data_sources(acs5)

initial acs5 shape: 3123
inital number of colums: 107
--------------------------------------------------------------
df shape after merging with tracts_place: 6566
df number of colums: 114
--------------------------------------------------------------
df shape after mergin with transit score: 2097
df number of colums: 126
--------------------------------------------------------------
df shape after mergin with jobs data: 2094
df number of colums: 129
--------------------------------------------------------------
df shape after grouping: 1716
df number of colums: 115
df cols omitted becauseo of grouping: {'index', 'tsplace_GEO_ID', 'tract_name', 'job_tract_label', 'ws_link', 'place_name', 'place_name_and_type', 'transit_summary', 'place_GEO_ID', 'job_tract_GEO_ID', 'transit_description', 'tract_GEO_ID', 'city_from_ts', 'geometry'}
df num columns with zero population: 0


In [9]:
tracts = gpd.read_file('shape_tracts/tl_2018_17_tract.shp')
tracts = tracts[['GEOID', 'NAMELSAD', 'ALAND', 'geometry']] \
                        .rename(columns={'GEOID': 'tract_GEO_ID', 'NAMELSAD': 'tract_name',
                       'ALAND': 'tract_area'})

In [10]:
tracts.head(4)

,tract_GEO_ID,tract_name,tract_area,geometry
0,17091011700,Census Tract 117,2370100,"POLYGON ((-87.88768 41.13594, -87.88764 41.136..."
1,17091011800,Census Tract 118,1790218,"POLYGON ((-87.89410 41.14388, -87.89400 41.143..."
2,17119400951,Census Tract 4009.51,5170038,"POLYGON ((-90.11192 38.70281, -90.11128 38.703..."
3,17119400952,Census Tract 4009.52,5751222,"POLYGON ((-90.09442 38.72031, -90.09360 38.720..."


In [19]:
tracts['centroid'] = tracts.geometry.centroid
tracts['centroid_lat'] = tracts.centroid.x
tracts['centroid_lng'] = tracts.centroid.y
tracts.head(4)

,tract_GEO_ID,tract_name,tract_area,geometry,centroid,centroid_lat,centroid_lng
0,17091011700,Census Tract 117,2370100,"POLYGON ((-87.88768 41.13594, -87.88764 41.136...",POINT (-87.87355 41.12949),-87.873553,41.129490
1,17091011800,Census Tract 118,1790218,"POLYGON ((-87.89410 41.14388, -87.89400 41.143...",POINT (-87.87646 41.13977),-87.876462,41.139775
2,17119400951,Census Tract 4009.51,5170038,"POLYGON ((-90.11192 38.70281, -90.11128 38.703...",POINT (-90.09829 38.72763),-90.098292,38.727634
3,17119400952,Census Tract 4009.52,5751222,"POLYGON ((-90.09442 38.72031, -90.09360 38.720...",POINT (-90.08180 38.72984),-90.081798,38.729842


In [32]:
from pandas.io.json import json_normalize
import requests 

def get_data(lat, lng):
    '''
    Gets data for one location.
    
    Inputs:
        lat, lng (str)
    Returns:
        (json)
    '''
    rv = None
    over_quota = False
    
    #key = '4c4eb18a1eea25128110eaf683aefab4'
    #key = 'ffd1c56f9abcf84872116b4cc2dfcf31' #Mike's key1
    key = '4c4eb18a1eea25128110eaf683aefab4' #Mike key2
    #key = 'ffd1c56f9abcf84872116b4cc2dfcf31' #Nathan key1
    #key = '4c4eb18a1eea25128110eaf683aefab4' #Nathan key2
    
    url = 'https://transit.walkscore.com/transit/score/?lat={}&lon={}&wsapikey={}'.format(lat, lng, key)
    
    r = requests.get(url)
    if not r.status_code==400:
        rv = r.json()
    if r.text == 'Over quota.':
        over_quota = True
        print('error: over quota')
    print(r)
    
    return rv, over_quota

In [33]:
def create_datalist(df):
    '''
    Turns list of jsons into a dataframe
    
    Inputs:
        acsse data frame (pandas Dataframe)
    Returns:
        list containing transit score for all places for which it is available (list of json)
    '''
    datalist = []
    over_quota_count = 0
    
    for row in df.itertuples():
        json, quota_bool = get_data(row.centroid_lat, row.centroid_lng)
        if not json is None:
            json['tract_GEO_ID'] = row.tract_GEO_ID
            json['centroid_lat'] = row.centroid_lat
            json['centroid_lng'] = row.centroid_lng
            datalist.append(json)
        if quota_bool:
            over_quota_count += 1
    
    return datalist, over_quota_count

In [34]:
datalist = create_datalist(tracts)

<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400

<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400

<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400

<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400

<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400

<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400

<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400]>
<Response [400

In [38]:
url = 'https://transit.walkscore.com/transit/score/?lat=41.9231&lon=-87.7093&wsapikey=ffd1c56f9abcf84872116b4cc2dfcf31'
r = requests.get(url)
r.text

'Over quota.'

In [39]:
'Over quota.' == 'Over quota.'

True